In [ ]:
!pip install pyjaspar
from pyjaspar import jaspardb
jdb_obj = jaspardb(release='JASPAR2024')
ids = ['MA0265.1', 'MA0265.2', 'MA0265.3']
motif = jdb_obj.fetch_motif_by_id(ids[0])

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╸ 51.0/51.0 MB 53.2 MB/s eta 0:00:01

In [ ]:
import numpy as np
import requests
import random

# Función para descargar un PWM de JASPAR usando su API con el formato de URL especificado
def get_pwm(jaspar_id):
    url = f'https://jaspar.elixir.no/api/v1/matrix/{jaspar_id}/'
    response = requests.get(url)
    if response.status_code == 200:
        pwm_data = response.json()['pfm']  # Extraer solo la matriz de frecuencias (pfm)
        return pwm_data
    else:
        raise Exception(f"No se pudo descargar el PWM de JASPAR para el ID {jaspar_id}")
for i in range(len(ids)):
  print(get_pwm(ids[i]))

In [ ]:
motifJ = get_pwm(ids[0])
t0 = [0.3, 0.2, 0.2, 0.3]

In [ ]:
motifJ
w = len(motifJ['A'])

Theta = np.zeros((w, 4))
for k in range(w):
  for nuc in motifJ:
    if nuc == 'A':
      Theta[k][0] = motifJ[nuc][k]
    if nuc == 'C':
      Theta[k][1] = motifJ[nuc][k]
    if nuc == 'G':
      Theta[k][2] = motifJ[nuc][k]
    if nuc == 'T':
      Theta[k][3] = motifJ[nuc][k]
Theta = 0.01*Theta
Theta

In [ ]:
#Creador de secuencias de ADN: A secuencias de largo L
def adninicial(A,L):
    M = np.random.choice(["A", "C", "G", "T"], size=(A,L))
    return M

adninicial(10,20)

In [ ]:
import numpy as np
import random

# Cálculo de la energía basada en el logaritmo de la razón de probabilidades
def calcular_energia_paper(S_i, a_i, Theta, theta_0, w):
    """
    Calcula la energía de una subsecuencia según la fórmula del paper.

    Args:
    - S_i (str): Secuencia donde se evalúa el motivo.
    - a_i (int): Posición inicial propuesta del motivo.
    - Theta (np.array): Matriz del modelo de motivos (tamaño w x 4).
    - theta_0 (list[float]): Modelo de fondo (probabilidades de A, C, G, T).
    - w (int): Longitud del motivo.

    Returns:
    - float: Energía de la subsecuencia en la posición propuesta.
    """
    K = "ACGT"
    subseq = S_i[a_i:a_i + w]
    energia = 0
    for j, nucleotide in enumerate(subseq):
        k = K.index(nucleotide)  # Índice del nucleótido
        prob_motif = Theta[j][k]
        prob_background = theta_0[k]
        energia -= np.log(prob_motif + (1.25/4) / prob_background)  # Agregar log-ratio
    return energia

# Función principal de Metropolis-Hastings con aceptación/rechazo basado en alpha_H
def metropolis_hastings_alpha_H(S, w, Theta, theta_0, n_iter):
    """
    Algoritmo de Metropolis-Hastings con criterio de aceptación basado en alpha_H.

    Args:
    - S (list[str]): Lista de secuencias.
    - w (int): Longitud del motivo.
    - Theta (np.array): Modelo de probabilidades del motivo (tamaño w x 4).
    - theta_0 (list[float]): Modelo de fondo (probabilidades de A, C, G, T).
    - n_iter (int): Número de iteraciones.

    Returns:
    - list[int]: Posiciones finales de los motivos (A).
    """
    N = len(S)
    L = [len(s) for s in S]  # Longitudes de las secuencias
    A = [random.randint(0, L[i] - w) for i in range(N)]  # Inicializar posiciones al azar

    for iteration in range(n_iter):
        # Elegir una secuencia al azar
        i = random.randint(0, N - 1)
        a_old = A[i]

        # Proponer nueva posición a_i'
        a_new = random.randint(0, L[i] - w)

        # Calcular energías para la propuesta y el estado actual
        E_old = calcular_energia_paper(S[i], a_old, Theta, theta_0, w)
        E_new = calcular_energia_paper(S[i], a_new, Theta, theta_0, w)

        # Calcular alpha_H (razón de aceptación)
        alpha_H = min(1, np.exp(E_old - E_new))

        # Aceptar o rechazar
        if random.uniform(0, 1) < alpha_H:
            A[i] = a_new  # Aceptar la nueva posición

        # (Opcional) Imprimir estado en cada iteración
        print(f"Iteración {iteration + 1}: A = {A}, E_old = {E_old:.3f}, E_new = {E_new:.3f}, alpha_H = {alpha_H:.3f}")

    return A

In [ ]:
S = adninicial(10,30)

In [ ]:
theta_0 = [0.3, 0.2, 0.2, 0.3]

n_iter = 10000  # Número de iteraciones

# Ejecutar el algoritmo
A_final = metropolis_hastings_alpha_H(S, w, Theta, t0, n_iter)

# Mostrar resultados
print("\nPosiciones finales A:", A_final)